In [809]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [810]:
def simu_success(tag, params):
    # which task
    if tag == "Asso-CR":
        path = "../Data/simuS1_group1_design.pkl"
        nitems = 96
        test1_num = 40
    elif tag == "Pair-CR":
        path = "../Data/simuS1_group2_design.pkl"
        nitems = 176
        test1_num = 80
    elif tag == "Item-CR":
        path = "../Data/simuS1_group3_design.pkl"
        nitems = 136
        test1_num = 80

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    # df_study = df_study.loc[df_study.session < 200]
    # df_test = df_test.loc[df_test.session < 200]

    # load semantic matrix
    s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test ,s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort

        tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
        corrid = np.searchsorted(tmp, tmp_corr)

        corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
        omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

        tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
        tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    return df_simu

In [811]:
def anal_perform(df_simu):

    # get correctness
    df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

    # recognition performance
    df_recog = df_simu.query("test==1")
    hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
    z_hr_far = sp.stats.norm.ppf(hr_far)
    print("recognition: \n", hr_far)
    print("d_prime: ", z_hr_far[1].item() - z_hr_far[0].item())

    # cued recall performance
    df_cr = df_simu.query("test==2")
    print("cued recall: \n", df_cr.correct.mean())

    # analyze pair
    def get_pair(df_tmp):
        df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
        df_tmp_pair.columns = ["test1","test2"]
        df_tmp_pair.reset_index(inplace=True)
        return df_tmp_pair

    df_simu_p = df_simu.query("pair_idx >= 0")
    df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
    df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize=False)
    print("contingency table: \n", df_tab)
    print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

    # compute" Q
    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)
    
    # print("Q: ", Yule_Q(df_tab[1][1],df_tab[0][1],df_tab[1][0],df_tab[0][0]))
    print("Q: ", Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5))  # add 0.5

    return None

### Study Params

In [812]:
# study params should be the same across three groups
params = cmr.make_default_params()
params.update(
    beta_enc = 0.3,
    gamma_fc = 0.2,
    gamma_cf = 0.2,
    s_fc = 0.2,
    s_cf = 0.2,
    phi_s = 1,
    phi_d = 0.6,
    d_ass = 1,
    learn_while_retrieving = True,
    var_enc = 0.7,
    bad_enc_ratio = 0.3,
)

In [813]:
params.update(
    beta_cue = 0.4,
    beta_rec = 0,
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.5,
    kappa = 0.01,
    lamb = 0.002,
    eta = 0.002,
    omega = 3,
    alpha = 1
)
params

{'beta_enc': 0.3,
 'beta_rec': 0,
 'beta_cue': 0.4,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 0.7,
 'bad_enc_ratio': 0.3,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

In [814]:
params_g1 = params.copy()
# params_g1.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.5,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g1

In [815]:
params_g2 = params.copy()
# params_g2.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g2

In [816]:
params_g3 = params.copy()
# params_g3.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g3

### Association - CR (Group1)

In [817]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params_g1)
df_simu_g1

CMR2 Time: 71.09446549415588


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,146,1542,1.0,0.010489,0.634740,1,BOLT,UTENSIL,1,0,34,-1.000000,-1.000000
1,0,147,1118,0.0,16.671247,0.266184,1,BOMB,PRAIRIE,1,1,0,-1.000000,-1.000000
2,0,1185,159,0.0,1.261297,0.395262,1,REPTILE,BOX,0,1,-1,-1.000000,-1.000000
3,0,358,1421,1.0,0.097278,0.523378,1,COMPASS,SWING,1,1,35,-1.000000,-1.000000
4,0,1201,1359,1.0,0.116974,0.514159,1,ROCK,SPRINKLE,1,0,12,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,999,1433,-1,1317.0,2640.000000,0.466015,2,TAPE,-1,155,0,26,-3.274151,-0.552003
59996,999,1463,-1,986.0,2680.000000,0.427666,2,TICK,-1,700,1,13,-0.708505,-0.588431
59997,999,990,-1,16.0,3060.000000,0.419290,2,OVAL,-1,1317,1,28,-3.124022,-0.572798
59998,999,695,-1,52.0,2670.000000,0.494629,2,HEAD,-1,52,0,19,-0.554971,-0.585605


In [818]:
csim_two = df_simu_g1.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.325846
1    0.427470
Name: csim, dtype: float64

In [819]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.3766579923195764

In [820]:
df_simu_g1['s_resp'] = df_simu_g1.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [821]:
anal_perform(df_simu_g1)

recognition: 
              Yes rate
correct_ans          
0              0.2712
1              0.6468
d_prime:  0.9858831240350268
cued recall: 
 0.274
contingency table: 
 test1   0.0   1.0
test2            
0.0    7052  7468
1.0      12  5468
contingency table norm: 
 test1     0.0     1.0
test2                
0.0    0.3526  0.3734
1.0    0.0006  0.2734
Q:  0.9951703898239399


### Pair - CR (Group2)

In [822]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params_g2)
df_simu_g2

CMR2 Time: 178.16244626045227


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1109,898,1.0,0.028645,0.584507,1,PORPOISE,MINT,1,1,32,-1.000000,-1.000000
1,0,661,567,0.0,23.437260,0.249152,1,GRILL,FINGER,0,-1,-1,-1.000000,-1.000000
2,0,1086,732,1.0,0.082826,0.531419,1,PLATE,ICING,1,1,39,-1.000000,-1.000000
3,0,249,720,0.0,1.574808,0.384162,1,CARROT,HOSE,1,1,13,-1.000000,-1.000000
4,0,1385,340,1.0,0.096813,0.523617,1,STONE,COD,1,0,28,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,934,-1,1059.0,3720.000000,0.443760,2,MUSHROOM,-1,1000,0,10,-0.640584,-0.598755
119996,999,1015,-1,288.0,4220.000000,0.499674,2,PARROT,-1,288,0,12,-0.535475,-0.562409
119997,999,1619,-1,141.0,3660.000000,0.494560,2,WOLF,-1,1185,1,36,-3.079216,-0.550750
119998,999,355,-1,1270.0,3640.000000,0.442354,2,COMMITTEE,-1,755,1,18,-3.211268,-0.604141


In [823]:
csim_two = df_simu_g2.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.248200
1    0.448983
Name: csim, dtype: float64

In [824]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.34859162230118107

In [825]:
df_simu_g2['s_resp'] = df_simu_g2.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [826]:
anal_perform(df_simu_g2)

recognition: 
              Yes rate
correct_ans          
0            0.079150
1            0.785525
d_prime:  2.201801890668122
cued recall: 
 0.214175
contingency table: 
 test1   0.0    1.0
test2             
0.0    8570  22863
1.0       9   8558
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.214250  0.571575
1.0    0.000225  0.213950
Q:  0.994095156609049


### Item - CR (Group3)

In [827]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params_g3)
df_simu_g3

CMR2 Time: 155.99832201004028


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1292,-1,1.0,1.605324,0.383202,1,SHORTCAKE,1,0,14,-1.000000,-1.000000
1,0,1052,-1,1.0,0.308513,0.465668,1,PERSON,1,0,39,-1.000000,-1.000000
2,0,135,-1,1.0,0.998423,0.406948,1,BLANKET,1,0,19,-1.000000,-1.000000
3,0,1365,-1,1.0,361.483795,0.112358,1,STAFF,0,-1,-1,-1.000000,-1.000000
4,0,760,-1,1.0,151.365845,0.155884,1,JOURNAL,0,-1,-1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1268,-1,-1.0,-1.000000,-1.000000,2,SENATOR,157,0,37,-3.187666,-0.630755
119996,999,537,-1,546.0,4120.000000,0.424323,2,ENVELOPE,546,1,3,-0.612201,-0.617936
119997,999,361,-1,-1.0,-1.000000,-1.000000,2,CONCRETE,260,0,28,-3.322301,-0.649163
119998,999,395,-1,356.0,3360.000000,0.477683,2,COWBOY,356,1,31,-0.567301,-0.648111


In [828]:
csim_two = df_simu_g3.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.236955
1    0.330744
Name: csim, dtype: float64

In [829]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.28384948908480945

In [830]:
df_simu_g3['s_resp'] = df_simu_g3.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [831]:
anal_perform(df_simu_g3)

recognition: 
              Yes rate
correct_ans          
0            0.266225
1            0.731025
d_prime:  1.2401863709859529
cued recall: 
 0.232675
contingency table: 
 test1    0.0    1.0
test2              
0.0    10303  20390
1.0      456   8851
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.257575  0.509750
1.0    0.011400  0.221275
Q:  0.8147787954402627


## No Variability

In [832]:
params.update(
    var_enc = 1,
    bad_enc_ratio = 1,
)
params

{'beta_enc': 0.3,
 'beta_rec': 0,
 'beta_cue': 0.4,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

In [833]:
params_g1 = params.copy()
# params_g1.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.5,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g1

In [834]:
params_g2 = params.copy()
# params_g2.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g2

In [835]:
params_g3 = params.copy()
# params_g3.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g3

### Association - CR (Group1)

In [836]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params_g1)
df_simu_g1

CMR2 Time: 74.22966647148132


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,146,1542,1.0,0.010489,0.634740,1,BOLT,UTENSIL,1,0,34,-1.000000,-1.000000
1,0,147,1118,0.0,1.457310,0.388039,1,BOMB,PRAIRIE,1,1,0,-1.000000,-1.000000
2,0,1185,159,0.0,1.545035,0.385116,1,REPTILE,BOX,0,1,-1,-1.000000,-1.000000
3,0,358,1421,1.0,0.105787,0.519185,1,COMPASS,SWING,1,1,35,-1.000000,-1.000000
4,0,1201,1359,1.0,0.119928,0.512912,1,ROCK,SPRINKLE,1,0,12,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,999,1433,-1,155.0,2110.000000,0.582220,2,TAPE,-1,155,0,26,-0.448275,-0.560545
59996,999,1463,-1,857.0,2570.000000,0.446323,2,TICK,-1,700,1,13,-0.499453,-0.563262
59997,999,990,-1,1317.0,2140.000000,0.581265,2,OVAL,-1,1317,1,28,-0.441209,-0.535834
59998,999,695,-1,1156.0,2730.000000,0.436382,2,HEAD,-1,52,0,19,-0.555298,-0.552143


In [837]:
csim_two = df_simu_g1.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.367408
1    0.504666
Name: csim, dtype: float64

In [838]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.43603700909763576

In [839]:
df_simu_g1['s_resp'] = df_simu_g1.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [840]:
anal_perform(df_simu_g1)

recognition: 
              Yes rate
correct_ans          
0             0.15850
1             0.88385
d_prime:  2.195096914173958
cued recall: 
 0.41665
contingency table: 
 test1   0.0   1.0
test2            
0.0    2169  9498
1.0     154  8179
contingency table norm: 
 test1      0.0      1.0
test2                  
0.0    0.10845  0.47490
1.0    0.00770  0.40895
Q:  0.847236499706565


### Pair - CR (Group2)

In [841]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params_g2)
df_simu_g2

CMR2 Time: 175.51242971420288


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1109,898,1.0,0.028645,0.584507,1,PORPOISE,MINT,1,1,32,-1.000000,-1.000000
1,0,661,567,0.0,23.437260,0.249152,1,GRILL,FINGER,0,-1,-1,-1.000000,-1.000000
2,0,1086,732,1.0,0.082826,0.531419,1,PLATE,ICING,1,1,39,-1.000000,-1.000000
3,0,249,720,1.0,0.052403,0.554308,1,CARROT,HOSE,1,1,13,-1.000000,-1.000000
4,0,1385,340,1.0,0.093647,0.525280,1,STONE,COD,1,0,28,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,934,-1,1181.0,2570.000000,0.487386,2,MUSHROOM,-1,1000,0,10,-3.277518,-0.510927
119996,999,1015,-1,1529.0,3250.000000,0.441330,2,PARROT,-1,288,0,12,-0.516394,-0.530803
119997,999,1619,-1,628.0,3290.000000,0.497672,2,WOLF,-1,1185,1,36,-3.103651,-0.512233
119998,999,355,-1,1135.0,3830.000000,0.421626,2,COMMITTEE,-1,755,1,18,-3.222997,-0.529300


In [842]:
csim_two = df_simu_g2.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.248298
1    0.518710
Name: csim, dtype: float64

In [843]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.38350435871999544

In [844]:
df_simu_g2['s_resp'] = df_simu_g2.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [845]:
anal_perform(df_simu_g2)

recognition: 
              Yes rate
correct_ans          
0            0.022275
1            0.975225
d_prime:  3.972707054086244
cued recall: 
 0.375725
contingency table: 
 test1  0.0    1.0
test2            
0.0    972  23999
1.0     19  15010
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.024300  0.599975
1.0    0.000475  0.375250
Q:  0.9378734206302972


### Item - CR (Group3)

In [846]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params_g3)
df_simu_g3

CMR2 Time: 135.31345224380493


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1292,-1,1.0,0.350559,0.459280,1,SHORTCAKE,1,0,14,-1.000000,-1.000000
1,0,1052,-1,1.0,0.283493,0.469897,1,PERSON,1,0,39,-1.000000,-1.000000
2,0,135,-1,1.0,0.871508,0.413745,1,BLANKET,1,0,19,-1.000000,-1.000000
3,0,1365,-1,1.0,354.443207,0.113341,1,STAFF,0,-1,-1,-1.000000,-1.000000
4,0,760,-1,1.0,148.892548,0.156707,1,JOURNAL,0,-1,-1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1268,-1,260.0,3130.000000,0.417788,2,SENATOR,157,0,37,-3.087209,-0.586574
119996,999,537,-1,-1.0,-1.000000,-1.000000,2,ENVELOPE,546,1,3,-0.606884,-0.607387
119997,999,361,-1,395.0,4310.000000,0.418409,2,CONCRETE,260,0,28,-3.165075,-0.606459
119998,999,395,-1,356.0,2660.000000,0.494929,2,COWBOY,356,1,31,-0.551679,-0.633629


In [847]:
csim_two = df_simu_g3.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.239780
1    0.365604
Name: csim, dtype: float64

In [848]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.30269186140199306

In [849]:
df_simu_g3['s_resp'] = df_simu_g3.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [850]:
anal_perform(df_simu_g3)

recognition: 
              Yes rate
correct_ans          
0             0.19490
1             0.83115
d_prime:  1.8186997653967534
cued recall: 
 0.414925
contingency table: 
 test1   0.0    1.0
test2             
0.0    5785  17618
1.0     969  15628
contingency table norm: 
 test1       0.0      1.0
test2                   
0.0    0.144625  0.44045
1.0    0.024225  0.39070
Q:  0.6822109031854448
